In [2]:
!pip install openai>=1.52.2
!pip install vllm>=0.6.3
!pip install triton>=3.1.0
!pip install apache-beam[gcp]==2.61.0
!pip install nest_asyncio # only needed in colab
!pip check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 21.3 MB/s et

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.
dask 2024.10.0 has requirement cloudpickle>=3.0.0, but you have cloudpickle 2.2.1.


## Colab only: allow nested asyncio

The vLLM model handler logic below uses asyncio to feed vLLM records. This only works if we are not already in an asyncio event loop. Most of the time, this is fine, but colab already operates in an event loop. To work around this, we can use nest_asyncio to make things work smoothly in colab. Do not include this step outside of colab.

In [3]:
# This should not be necessary outside of colab.
import nest_asyncio
nest_asyncio.apply()


In [ ]:
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.vllm_inference import VLLMCompletionsModelHandler
from apache_beam.ml.inference.base import PredictionResult
import apache_beam as beam

model_handler = VLLMCompletionsModelHandler('facebook/opt-125m')

INFO 02-07 02:51:01 __init__.py:190] Automatically detected platform cuda.


Next, define examples to run inference against, and define a helper function to print out the inference results.

In [ ]:
class FormatOutput(beam.DoFn):
  def process(self, element, *args, **kwargs):
    yield "Input: {input}, Output: {output}".format(input=element.example, output=element.inference)

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "Emperor penguins are",
]

In [ ]:
class FormatOutput(beam.DoFn):
  def process(self, element, *args, **kwargs):
    yield "Input: {input}, Output: {output}".format(input=element.example, output=element.inference)

prompts = [
    "1+3 is",
]

In [ ]:
with beam.Pipeline() as p:
  _ = (p | beam.Create(prompts) # Create a PCollection of the prompts.
         | RunInference(model_handler) # Send the prompts to the model and get responses.
         | beam.ParDo(FormatOutput()) # Format the output.
         | beam.Map(print) # Print the formatted output.
  )

ERROR:apache_beam.runners.common:Failed to start vLLM server, polling process exited with code 1.  Next time a request is tried, the server will be restarted [while running '[14]: RunInference/BeamML_RunInference']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1565, in apache_beam.runners.common.DoFnRunner._invoke_lifecycle_method
  File "apache_beam/runners/common.py", line 607, in apache_beam.runners.common.DoFnInvoker.invoke_setup
  File "/usr/local/lib/python3.11/dist-packages/apache_beam/ml/inference/base.py", line 1746, in setup
    self._model = self._load_model()
                  ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/apache_beam/ml/inference/base.py", line 1709, in _load_model
    load, tag=copy_tag, always_proxy=True).acquire())
                                           ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/apache_beam/utils/multi_process_shared.py", line 303, in acquire
    singleton = self._

Exception: Failed to start vLLM server, polling process exited with code 1.  Next time a request is tried, the server will be restarted [while running '[14]: RunInference/BeamML_RunInference']

Finally, run the pipeline.

This step might take a minute or two, because the model needs to download before Apache Beam can start running inference.

In [ ]:
!pip install apache-beam[interactive]

In [ ]:
with beam.Pipeline() as p:
  _ = (p | beam.Create(prompts) # Create a PCollection of the prompts.
         | RunInference(model_handler) # Send the prompts to the model and get responses.
         | beam.ParDo(FormatOutput()) # Format the output.
         | beam.Map(print) # Print the formatted output.
  )

Input: Hello, my name is, Output: Completion(id='cmpl-1dc32d984db145cfba021ba56a8ba320', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' Joel, my dad is Anton Harriman and my wife is Lydia. ', stop_reason=None, prompt_logprobs=None)], created=1738896794, model='facebook/opt-125m', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=6, total_tokens=22, completion_tokens_details=None, prompt_tokens_details=None))
Input: The president of the United States is, Output: Completion(id='cmpl-a527d10924654342ac2a0d1cea351e8d', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' using a woman....in Latino voices.\n>*but Latino voices* ', stop_reason=None, prompt_logprobs=None)], created=1738896794, model='facebook/opt-125m', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=8, total_tokens=24, completion_tokens_details=N

In [ ]:
from vllm import LLM
model = LLM(model="facebook/opt-125m")
output = model.generate("My name is")

INFO 02-07 03:09:05 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-07 03:09:21 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 02-07 03:09:21 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=facebook/opt-125m, num_scheduler_steps=1, multi_ste

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-07 03:09:25 model_runner.py:1115] Loading model weights took 0.2389 GB
INFO 02-07 03:09:26 worker.py:267] Memory profiling takes 0.68 seconds
INFO 02-07 03:09:26 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 03:09:26 worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 02-07 03:09:26 executor_base.py:110] # CUDA blocks: 22813, # CPU blocks: 7281
INFO 02-07 03:09:26 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 178.23x
INFO 02-07 03:09:30 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:28<00:00,  1.24it/s]

INFO 02-07 03:09:58 model_runner.py:1562] Graph capturing finished in 28 secs, took 0.14 GiB
INFO 02-07 03:09:58 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 33.48 seconds



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.40it/s, est. speed input: 46.05 toks/s, output: 184.13 toks/s]


In [ ]:
print(output)

[RequestOutput(request_id=0, prompt='My name is', prompt_token_ids=[2, 2387, 766, 16], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Joel, my dad is Anton Harrap, and my wife is Lydia. ', token_ids=(8966, 6, 127, 4252, 16, 12929, 2482, 8645, 6, 8, 127, 1141, 16, 28301, 4, 1437), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738897799.5141833, last_token_time=1738897799.6022253, first_scheduled_time=1738897799.5183554, first_token_time=1738897799.5404072, time_in_queue=0.004172086715698242, finished_time=1738897799.6024256, scheduler_time=0.0025901450003402715, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [4]:
from vllm import LLM
model = LLM(model="facebook/opt-125m")
output = model.generate("My name is")
print(output)

INFO 02-07 18:44:18 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

INFO 02-07 18:44:38 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 02-07 18:44:38 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=facebook/opt-125m, num_scheduler_steps=1, multi_ste

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

INFO 02-07 18:44:49 model_runner.py:1115] Loading model weights took 0.2389 GB
INFO 02-07 18:44:50 worker.py:267] Memory profiling takes 0.99 seconds
INFO 02-07 18:44:50 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 18:44:50 worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 02-07 18:44:50 executor_base.py:110] # CUDA blocks: 22813, # CPU blocks: 7281
INFO 02-07 18:44:50 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 178.23x
INFO 02-07 18:44:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:29<00:00,  1.20it/s]

INFO 02-07 18:45:25 model_runner.py:1562] Graph capturing finished in 29 secs, took 0.14 GiB
INFO 02-07 18:45:25 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 36.37 seconds



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.95it/s, est. speed input: 47.92 toks/s, output: 191.59 toks/s]

[RequestOutput(request_id=0, prompt='My name is', prompt_token_ids=[2, 2387, 766, 16], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Joel, my dad is Anton Harrap, and my wife is Lydia. ', token_ids=(8966, 6, 127, 4252, 16, 12929, 2482, 8645, 6, 8, 127, 1141, 16, 28301, 4, 1437), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738953926.2798507, last_token_time=1738953926.3722303, first_scheduled_time=1738953926.2917411, first_token_time=1738953926.310151, time_in_queue=0.011890411376953125, finished_time=1738953926.372415, scheduler_time=0.0034410060000027443, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [ ]:
from vllm import LLM, SamplingParams

INFO 02-07 03:06:02 __init__.py:190] Automatically detected platform cuda.


In [ ]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [ ]:
llm = LLM(model="facebook/opt-125m")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-07 03:06:24 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
INFO 02-07 03:06:24 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=facebook/opt-125m, num_scheduler_steps=1, multi_ste

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-07 03:06:28 model_runner.py:1115] Loading model weights took 0.2389 GB
INFO 02-07 03:06:29 worker.py:267] Memory profiling takes 0.81 seconds
INFO 02-07 03:06:29 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 03:06:29 worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 02-07 03:06:30 executor_base.py:110] # CUDA blocks: 22813, # CPU blocks: 7281
INFO 02-07 03:06:30 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 178.23x
INFO 02-07 03:06:33 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:29<00:00,  1.19it/s]

INFO 02-07 03:07:03 model_runner.py:1562] Graph capturing finished in 29 secs, took 0.14 GiB
INFO 02-07 03:07:03 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 34.54 seconds


In [ ]:
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 21.89it/s, est. speed input: 142.38 toks/s, output: 350.44 toks/s]

Prompt: 'Hello, my name is', Generated text: " Joel.  I'm a homemaker.  I drink tea a lot and"
Prompt: 'The president of the United States is', Generated text: ' giving his blessing to new "master" food manufacturing technology that could revolutionize food'
Prompt: 'The capital of France is', Generated text: ' now the capital of the French Resistance\nThis is a really bad troll.'
Prompt: 'The future of AI is', Generated text: ' now at a crossroads\nFor now, the gap between AI-based solutions'


In [ ]:
! huggingface-cli login

To run the pipeline in Apache Beam, run the following code. Update the `VLLMCompletionsModelHandler` object with the new parameters, which match the command from the previous cell. Reuse all of the pipeline logic from the previous pipelines.

In [1]:
from vllm import LLM
model = LLM(model="openai-community/gpt2")
output = model.generate("who is ai?")
print(output)

INFO 02-07 18:47:17 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

INFO 02-07 18:47:21 config.py:2382] Downcasting torch.float32 to torch.float16.
INFO 02-07 18:47:33 config.py:542] This model supports multiple tasks: {'embed', 'reward', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 02-07 18:47:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='openai-community/gpt2', speculative_config=None, tokenizer='openai-community/gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_ti

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

INFO 02-07 18:47:49 model_runner.py:1115] Loading model weights took 0.2378 GB
INFO 02-07 18:47:50 worker.py:267] Memory profiling takes 0.72 seconds
INFO 02-07 18:47:50 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 18:47:50 worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 12.53GiB.
INFO 02-07 18:47:50 executor_base.py:110] # CUDA blocks: 22815, # CPU blocks: 7281
INFO 02-07 18:47:50 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 356.48x
INFO 02-07 18:47:54 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:28<00:00,  1.23it/s]

INFO 02-07 18:48:22 model_runner.py:1562] Graph capturing finished in 28 secs, took 0.14 GiB
INFO 02-07 18:48:22 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 33.60 seconds



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.26it/s, est. speed input: 37.00 toks/s, output: 118.39 toks/s]

[RequestOutput(request_id=0, prompt='who is ai?', prompt_token_ids=[8727, 318, 257, 72, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Visit this page ...olla?: https://yubiverse.wordpress.com/', token_ids=(16440, 428, 2443, 2644, 33011, 27514, 3740, 1378, 88, 29603, 4399, 13, 40346, 13, 785, 14), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738954104.0973358, last_token_time=1738954104.2458417, first_scheduled_time=1738954104.1138325, first_token_time=1738954104.1495898, time_in_queue=0.016496658325195312, finished_time=1738954104.2460196, scheduler_time=0.002252922999900875, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [2]:
!huggingface-cli login




    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushi

In [1]:
from vllm import LLM
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct")
output = model.generate("who is ai?")
print(output)

INFO 02-07 18:51:05 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

INFO 02-07 18:51:22 config.py:542] This model supports multiple tasks: {'generate', 'score', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 02-07 18:51:22 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-07 18:51:22 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-07 18:51:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, p

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

INFO 02-07 18:51:28 cuda.py:179] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 02-07 18:51:28 cuda.py:227] Using XFormers backend.


ValueError: Bfloat16 is only supported on GPUs with compute capability of at least 8.0. Your Tesla T4 GPU has compute capability 7.5. You can use float16 instead by explicitly setting the`dtype` flag in CLI, for example: --dtype=half.

In [2]:
from vllm import LLM

# Instantiate the model with dtype='half' to use float16
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct", dtype="half")
output = model.generate("who is ai?")
print(output)

WARNING 02-07 18:52:25 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 18:52:26 config.py:542] This model supports multiple tasks: {'generate', 'score', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 02-07 18:52:26 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-07 18:52:26 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-07 18:52:26 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=13

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 18:54:56 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 18:54:58 worker.py:267] Memory profiling takes 1.27 seconds
INFO 02-07 18:54:58 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 18:54:58 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 6.02GiB.
INFO 02-07 18:54:58 executor_base.py:110] # CUDA blocks: 3523, # CPU blocks: 2340
INFO 02-07 18:54:58 executor_base.py:115] Maximum concurrency for 131072 tokens per request: 0.43x


ValueError: The model's max seq len (131072) is larger than the maximum number of tokens that can be stored in KV cache (56368). Try increasing `gpu_memory_utilization` or decreasing `max_model_len` when initializing the engine.

In [1]:
from vllm import LLM

# Increase gpu_memory_utilization to allocate more GPU memory to the KV cache.
# You may need to adjust this value based on your available GPU memory.
# Decrease max_model_len if you don't need the full context window.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.9,  # Increase memory utilization
            max_model_len=65536)        # Decrease max sequence length (optional)

output = model.generate("who is ai?")
print(output)

INFO 02-07 18:56:21 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 18:56:25 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 18:56:37 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 02-07 18:56:37 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-07 18:56:37 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-07 18:56:37 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=65

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 18:57:05 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 18:57:06 worker.py:267] Memory profiling takes 1.27 seconds
INFO 02-07 18:57:06 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 18:57:06 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 6.02GiB.
INFO 02-07 18:57:07 executor_base.py:110] # CUDA blocks: 3523, # CPU blocks: 2340
INFO 02-07 18:57:07 executor_base.py:115] Maximum concurrency for 65536 tokens per request: 0.86x


ValueError: The model's max seq len (65536) is larger than the maximum number of tokens that can be stored in KV cache (56368). Try increasing `gpu_memory_utilization` or decreasing `max_model_len` when initializing the engine.

In [ ]:
use_cached_outputs=False,

### aشغال

In [1]:
from vllm import LLM

# Increase gpu_memory_utilization to allocate more GPU memory to the KV cache.
# You may need to adjust this value based on your available GPU memory.
# Decrease max_model_len if you don't need the full context window.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.9,  # Increase memory utilization
            max_model_len=56368)        # Decrease max sequence length (optional)

output = model.generate("who is ai?")
print(output)

INFO 02-07 18:59:04 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 18:59:08 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 18:59:22 config.py:542] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
WARNING 02-07 18:59:22 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-07 18:59:22 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-07 18:59:22 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=56

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 18:59:48 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 18:59:50 worker.py:267] Memory profiling takes 1.29 seconds
INFO 02-07 18:59:50 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 18:59:50 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 6.02GiB.
INFO 02-07 18:59:50 executor_base.py:110] # CUDA blocks: 3523, # CPU blocks: 2340
INFO 02-07 18:59:50 executor_base.py:115] Maximum concurrency for 56368 tokens per request: 1.00x
INFO 02-07 18:59:57 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:36<00:00,  1.05s/it]

INFO 02-07 19:00:33 model_runner.py:1562] Graph capturing finished in 37 secs, took 0.19 GiB
INFO 02-07 19:00:34 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 45.29 seconds



Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.89s/it, est. speed input: 0.56 toks/s, output: 1.80 toks/s]

[RequestOutput(request_id=0, prompt='who is ai?', prompt_token_ids=[128000, 14965, 374, 16796, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' \nAI, or Artificial Intelligence, refers to the development of computer systems that can', token_ids=(720, 15836, 11, 477, 59294, 22107, 11, 19813, 311, 279, 4500, 315, 6500, 6067, 430, 649), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738954834.1211886, last_token_time=1738954843.0011997, first_scheduled_time=1738954834.1440115, first_token_time=1738954842.574161, time_in_queue=0.022822856903076172, finished_time=1738954843.0013778, scheduler_time=0.0019335480008066952, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [1]:
from vllm import LLM

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=4096,         # Further decrease max sequence length
            swap_space=20)              # Increase swap space (in GB) to handle larger contexts

output = model.generate("who is ai?")
print(output)

INFO 02-07 19:02:57 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 19:03:02 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:03:18 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'score', 'generate', 'classify'}. Defaulting to 'generate'.


ValueError: Too large swap space. 20.00 GiB out of the 12.67 GiB total CPU memory is allocated for the swap space.

In [1]:
from vllm import LLM

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=4096,         # Further decrease max sequence length
            swap_space=10)              # Increase swap space (in GB) to handle larger contexts

output = model.generate("who is ai?")
print(output)

INFO 02-07 19:04:32 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 19:04:36 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:04:48 config.py:542] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed', 'score'}. Defaulting to 'generate'.


ValueError: Too large swap space. 10.00 GiB out of the 12.67 GiB total CPU memory is allocated for the swap space.

### شغال

In [2]:
from vllm import LLM

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=4096,         # Further decrease max sequence length
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts

output = model.generate("who is ai?")
print(output)

WARNING 02-07 19:05:59 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:06:00 config.py:542] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
INFO 02-07 19:06:00 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, co

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 19:06:33 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 19:06:35 worker.py:267] Memory profiling takes 1.91 seconds
INFO 02-07 19:06:35 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 19:06:35 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 6.75GiB.
INFO 02-07 19:06:36 executor_base.py:110] # CUDA blocks: 3947, # CPU blocks: 1755
INFO 02-07 19:06:36 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 15.42x
INFO 02-07 19:06:40 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:34<00:00,  1.03it/s]

INFO 02-07 19:07:14 model_runner.py:1562] Graph capturing finished in 34 secs, took 0.19 GiB
INFO 02-07 19:07:14 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 40.63 seconds



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s, est. speed input: 8.92 toks/s, output: 28.55 toks/s]

[RequestOutput(request_id=0, prompt='who is ai?', prompt_token_ids=[128000, 14965, 374, 16796, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' \nAI, or Artificial Intelligence, refers to the development of computer systems that can', token_ids=(720, 15836, 11, 477, 59294, 22107, 11, 19813, 311, 279, 4500, 315, 6500, 6067, 430, 649), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738955234.3110228, last_token_time=1738955234.850597, first_scheduled_time=1738955234.3175824, first_token_time=1738955234.412715, time_in_queue=0.006559610366821289, finished_time=1738955234.850858, scheduler_time=0.004167663000089306, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


In [ ]:
prompt = "who is ai? Be creative and imaginative in your response."
output = model.generate(prompt, sampling_params=sampling_params)
print(output)

In [ ]:
sampling_params = SamplingParams(temperature=0.9, top_p=0.95, max_tokens=50)  # Increased temperature
output = model.generate("who is ai?", sampling_params=sampling_params)
print(output)

In [ ]:
from vllm import LLM, SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.9,
            max_model_len=56368)

# Set the maximum number of new tokens to 50
sampling_params = SamplingParams(max_tokens=50)

output = model.generate("who is ai?", sampling_params=sampling_params)
print(output)

In [ ]:
from vllm import LLM, SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.9,
            max_model_len=56368)

# Enable random sampling with temperature and top_p
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=50)

output = model.generate("who is ai?", sampling_params=sampling_params)
print(output)

In [ ]:
from vllm import LLM, SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.9,
            max_model_len=56368)

# Enable random sampling with temperature
sampling_params = SamplingParams(temperature=0.8, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("who is ai?", sampling_params=sampling_params)
print(output)

In [1]:
from vllm import LLM

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=4096,         # Further decrease max sequence length
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Write a story about a cat", sampling_params=sampling_params)

print(output)

INFO 02-07 19:10:10 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 19:10:14 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:10:32 config.py:542] This model supports multiple tasks: {'generate', 'score', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 02-07 19:10:32 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, co

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 19:11:03 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 19:11:05 worker.py:267] Memory profiling takes 1.89 seconds
INFO 02-07 19:11:05 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 19:11:05 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 6.75GiB.
INFO 02-07 19:11:05 executor_base.py:110] # CUDA blocks: 3947, # CPU blocks: 1755
INFO 02-07 19:11:05 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 15.42x
INFO 02-07 19:11:09 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:31<00:00,  1.12it/s]

INFO 02-07 19:11:40 model_runner.py:1562] Graph capturing finished in 31 secs, took 0.19 GiB
INFO 02-07 19:11:40 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 37.17 seconds


NameError: name 'SamplingParams' is not defined

### شغال

In [1]:
from vllm import LLM, SamplingParams # Import SamplingParams

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=4096,         # Further decrease max sequence length
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Write a story about a cat", sampling_params=sampling_params)

print(output)

INFO 02-07 19:12:32 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 19:12:36 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:12:54 config.py:542] This model supports multiple tasks: {'score', 'embed', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 02-07 19:12:54 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, co

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 19:13:27 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 19:13:29 worker.py:267] Memory profiling takes 1.82 seconds
INFO 02-07 19:13:29 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 19:13:29 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 6.75GiB.
INFO 02-07 19:13:29 executor_base.py:110] # CUDA blocks: 3947, # CPU blocks: 1755
INFO 02-07 19:13:29 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 15.42x
INFO 02-07 19:13:32 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:32<00:00,  1.07it/s]

INFO 02-07 19:14:05 model_runner.py:1562] Graph capturing finished in 33 secs, took 0.19 GiB
INFO 02-07 19:14:05 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 38.37 seconds



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.67s/it, est. speed input: 4.21 toks/s, output: 30.04 toks/s]

[RequestOutput(request_id=0, prompt='Write a story about a cat', prompt_token_ids=[128000, 8144, 264, 3446, 922, 264, 8415], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' who dreams of becoming a great singer\nWhiskers, a sleek black cat with a spark in her eye, had always been fascinated by the sounds of the city. She would sit for hours on the windowsill, listening to the wail of', token_ids=(889, 19226, 315, 10671, 264, 2294, 23597, 198, 1671, 3267, 388, 11, 264, 48494, 3776, 8415, 449, 264, 15541, 304, 1077, 8071, 11, 1047, 2744, 1027, 61914, 555, 279, 10578, 315, 279, 3363, 13, 3005, 1053, 2503, 369, 4207, 389, 279, 11276, 484, 11, 14624, 311, 279, 289, 607, 315), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738955645.742075, last_token_time=1738955647.3974645, first_scheduled_time=1738955645.7614226, first_token_time=173895


الحل:

تشغيل النموذج في وضع eager:
قم بتعيين enforce_eager=True عند تهيئة VLLMCompletionsModelHandler:

model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', enforce_eager=True)
Use code with caution
أو استخدم --enforce-eager في سطر الأوامر عند تشغيل البرنامج النصي.
تقليل استخدام ذاكرة GPU:
قم بتقليل gpu_memory_utilization عند تهيئة VLLMCompletionsModelHandler:

model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', gpu_memory_utilization=0.8)
Use code with caution
تقليل max_num_seqs:
قم بتقليل max_num_seqs عند تهيئة VLLMCompletionsModelHandler:

model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', max_num_seqs=16)
Use code with caution
الرسائل:

INFO 02-07 19:14:05 model_runner.py:1562] Graph capturing finished in 33 secs, took 0.19 GiB: تشير هذه الرسالة إلى أن التقاط CUDA graph قد اكتمل بنجاح.
INFO 02-07 19:14:05 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 38.37 seconds: تشير هذه الرسالة إلى أن تهيئة محرك vLLM قد اكتملت بنجاح.
التفسير:

يشير التحذير إلى أن CUDA graphs قد لا تكون مناسبة للنموذج الذي تستخدمه، وقد يؤدي ذلك إلى نتائج غير متوقعة.
تم التقاط CUDA graph بنجاح، لكن قد تحتاج إلى معالجة التحذير لضمان الأداء الصحيح للنموذج.
تم تهيئة محرك vLLM بنجاح.
التوصية:

جرب أولاً تشغيل النموذج في وضع eager (enforce_eager=True) لمعرفة ما إذا كان هذا يحل المشكلة.
إذا استمرت المشكلة، فحاول تقليل استخدام ذاكرة GPU أو تقليل max_num_seqs.




التوصية:

جرب أولاً تشغيل النموذج في وضع eager (enforce_eager=True) لمعرفة ما إذا كان هذا يحل المشكلة.
إذا استمرت المشكلة، فحاول تقليل استخدام ذاكرة GPU أو تقليل

In [ ]:
model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', enforce_eager=True)

In [ ]:
model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', gpu_memory_utilization=0.8)

In [ ]:
model_handler = VLLMCompletionsModelHandler('facebook/opt-125m', max_num_seqs=16)

int=1, dtype: str='auto', quantization: Optional[str]=None, revision: Optional[str]=None, tokenizer_revision: Optional[str]=None, seed: int=0, gpu_memory_utilization: float=0.9, swap_space: float=4, cpu_offload_gb: float=0, enforce_eager: Optional[bool]=None, max_seq_len_to_capture: int=8192, disable_custom_all_reduce: bool=False, disable_async_output_proc: bool=False, hf_overrides: Optional[HfOverrides]=None, mm_processor_kwargs: Optional[Dict[str, Any]]=None, task: TaskOption='auto', override_pooler_config: Optional[PoolerConfig]=None, compilation_config: Optional[Un

In [ ]:
from vllm import LLM, SamplingParams # Import SamplingParams

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            quantization=8
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Write a story about a cat", sampling_params=sampling_params)

print(output)

In [ ]:
!pip install bitsandbytes
!pip install autoawq

In [17]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.5 MB/s eta 0:00:00
  Created wheel for autoawq: filename=autoawq-0.2.8-py3-none-any.whl size=108744 sha256=c47121b5b1d89a61285f430c3e4bec9f1ceedf147efc24cfef8b5f0c98d1e941
  Stored in directory: /root/.cache/pip/wheels/fd/03/fe/99c1c678bfe8aca712186466969ed866f52feda95ae1dcd1b1
Successfully built autoawq
  Attempting uninstall: fss

In [ ]:
import bitsandbytes as bnb
# or
from awq import AutoAWQForCausalLM

In [ ]:
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            quantization="bnb",  # أو "awq" أو "gptq"
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)

In [ ]:
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            quantization="bnb",
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)

In [ ]:
from awq import AutoAWQForCausalLM

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="half",
            quantization="bnb",  # أو "awq" أو "gptq"
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)

حسناً، سأشرح لك معنى dtype="half" والبدائل المتاحة لتغييره:

معنى dtype="half":

يشير dtype="half" إلى نوع بيانات الفاصلة العائمة نصف الدقة (FP16). هذا يعني أن النموذج سيستخدم أعدادًا فاصلة عائمة بدقة 16 بت، مما يقلل من استخدام الذاكرة ويحسن الأداء. يعتبر dtype="half" مناسبًا لأجهزة مثل وحدات معالجة الرسومات (GPU) التي تم تحسينها لعمليات FP16.

بدائل dtype="half":

تتوفر بعض بدائل dtype="half"، اعتمادًا على متطلباتك:

dtype="float" (أو dtype="float32"): يحدد هذا نوع بيانات الفاصلة العائمة أحادية الدقة (FP32). يوفر دقة أعلى من dtype="half" لكنه يستهلك ذاكرة أكثر.

dtype="bfloat16": يحدد هذا نوع بيانات الفاصلة العائمة مع نطاق أوسع من dtype="half" لكن بدقة أقل. يمكن أن يكون مفيدًا لبعض النماذج التي تتطلب نطاقًا أكبر.

dtype="auto": سيحاول vLLM تلقائيًا تحديد نوع البيانات المناسب بناءً على النموذج والجهاز المستخدم. غالبًا ما يكون هذا الخيار مناسبًا إذا لم تكن متأكدًا من نوع البيانات الذي يجب استخدامه.

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.0 MB/s eta 0:00:00


INFO 02-07 19:34:18 __init__.py:190] Automatically detected platform cuda.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
WARNING 02-07 19:34:23 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:34:37 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-1-8a93e10773b9> in <cell line: 0>()
      3
      4 # Further reduce max_model_len and try increasing swap space.
----> 5 model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
      6             quantization="bnb",
      7             dtype="half",

6 frames
/usr/local/lib/python3.11/dist-packages/vllm/config.py in _verify_quantization(self)
    613         if self.quantization is not None:
    614             if self.quantization not in supported_quantization:
--> 615                 raise ValueError(
    616                     f"Unknown quantization method: {self.quantization}. Must "
    617                     f"be one of {supported_quantization}.")

ValueError: Unknown quantization method: bnb. Must be one of ['aqlm', 'awq', 'deepspeedfp', 'tpu_int8', 'fp8', 'fbgemm_fp8', 'modelopt', 'marlin', 'gguf', 'gptq_marlin_24', 'gptq_marlin', 'awq_marlin', 'gptq', 'compressed-tensors', 'bitsandbytes', 'qqq', 'hqq', 'experts_int8', 'neuron_quant', 'ipex', 'quark', 'moe_wna16'].


ترتيب أساليب التكميم من الأكبر إلى الأصغر من حيث استخدام الذاكرة:
بشكل عام، يصعب تحديد ترتيب دقيق لأساليب التكميم من حيث استخدام الذاكرة لأنها تعتمد على عوامل متعددة مثل بنية النموذج وحجم البيانات ونوع المهمة. ومع ذلك، يمكننا تقديم ترتيب تقريبي يعتمد على خصائص كل طريقة:

استخدام الذاكرة الأعلى:

float32 (بدون تكميم): يستخدم أعلى مستوى من الذاكرة نظرًا لعدم تكميم الأوزان أو التنشيطات.
bfloat16: يقدم توازناً بين الدقة واستخدام الذاكرة، ولكنه لا يزال يستهلك ذاكرة أكثر من أساليب التكميم الأخرى.
fp8: توفر دقة جيدة لكنها تتطلب ذاكرة أكثر من أساليب التكميم الأكثر عدوانية.
استخدام الذاكرة المتوسط:

aqlm, awq, gptq: هذه الأساليب تقدم توازناً بين الدقة والضغط، وتستخدم ذاكرة أقل من الأنواع السابقة.
deepspeedfp, tpu_int8: تم تصميمها للاستخدام على أجهزة معينة (Deepspeed و TPU) ويمكن أن تقدم أداءً جيدًا مع تقليل استخدام الذاكرة.
استخدام الذاكرة الأقل:

fbgemm_fp8, modelopt, marlin: توفر ضغطًا عاليًا مع تقليل كبير في استخدام الذاكرة، لكن قد تؤثر على الدقة.
gguf, gptq_marlin_24, gptq_marlin, awq_marlin: تستخدم تكميمًا متقدمًا لتحقيق ضغط أعلى مع الحفاظ على دقة مقبولة.
compressed-tensors, bitsandbytes, qqq, hqq, experts_int8, neuron_quant, ipex, quark, moe_wna16: تستخدم تكميمًا أكثر عدوانية أو تقنيات محددة لتحقيق الحد الأقصى من ضغط النموذج.
ملاحظة: هذا الترتيب تقريبي وقد يختلف حسب النموذج وحجم البيانات.

توصية:

إذا كنت محدودًا في الذاكرة، فابدأ باستخدام أساليب التكميم الأكثر عدوانية (مثل compressed-tensors أو bitsandbytes).
إذا كنت بحاجة إلى دقة عالية، فاستخدم أساليب تكميم أقل عدوانية (مثل fp8 أو aqlm).
جرّب خيارات التكميم المختلفة لمعرفة ما يناسب نموذجك ومهمتك بشكل أفضل.

In [1]:
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            quantization="bnb",
            dtype="half",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)


INFO 02-07 19:34:18 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 19:34:23 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 19:34:37 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.


ValueError: Unknown quantization method: bnb. Must be one of ['aqlm', 'awq', 'deepspeedfp', 'tpu_int8', 'fp8', 'fbgemm_fp8', 'modelopt', 'marlin', 'gguf', 'gptq_marlin_24', 'gptq_marlin', 'awq_marlin', 'gptq', 'compressed-tensors', 'bitsandbytes', 'qqq', 'hqq', 'experts_int8', 'neuron_quant', 'ipex', 'quark', 'moe_wna16'].

In [ ]:
dtype="half",

In [16]:
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

# Further reduce max_model_len and try increasing swap space.
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            quantization="auto",
            dtype="bfloat16",
            gpu_memory_utilization=0.95,  # Increase memory utilization further if possible
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)              # Increase swap space (in GB) to handle larger contexts
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)


INFO 02-07 19:45:41 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.


ValueError: Unknown quantization method: auto. Must be one of ['aqlm', 'awq', 'deepspeedfp', 'tpu_int8', 'fp8', 'fbgemm_fp8', 'modelopt', 'marlin', 'gguf', 'gptq_marlin_24', 'gptq_marlin', 'awq_marlin', 'gptq', 'compressed-tensors', 'bitsandbytes', 'qqq', 'hqq', 'experts_int8', 'neuron_quant', 'ipex', 'quark', 'moe_wna16'].

In [ ]:
model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="float",  # أو "bfloat16" أو "auto"
            # ... rest of the parameters ...
)

In [ ]:
!pip install bitsandbytes
!pip install autoawq

إذا كنت تستخدم gptq ، تأكد من تثبيت triton : !pip install triton


### ayhشغال

In [2]:
from awq import AutoAWQForCausalLM
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="float16",
            #quantization="",  # أو "awq" أو "gptq"
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=16,
            swap_space=3)
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

WARNING 02-07 20:03:39 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 20:03:39 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'reward', 'score', 'generate'}. Defaulting to 'generate'.
WARNING 02-07 20:03:39 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 20:03:39 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 20:03:39 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disa

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 20:04:12 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 20:04:14 worker.py:267] Memory profiling takes 1.47 seconds
INFO 02-07 20:04:14 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 20:04:14 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.14GiB; the rest of the memory reserved for KV Cache is 7.80GiB.
INFO 02-07 20:04:14 executor_base.py:110] # CUDA blocks: 4566, # CPU blocks: 1755
INFO 02-07 20:04:14 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 35.67x
INFO 02-07 20:04:17 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 5.71 seconds


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.81s/it, est. speed input: 4.44 toks/s, output: 27.72 toks/s]

[RequestOutput(request_id=0, prompt='Who is Napoleon Bonaparte?', prompt_token_ids=[128000, 15546, 374, 70527, 13789, 391, 20430, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution and became Emperor of France in 1804. He is considered one of the greatest military minds in history, known for his tactical genius and strategic leadership.\n', token_ids=(70527, 13789, 391, 20430, 574, 264, 8753, 6411, 323, 5054, 7808, 889, 16392, 311, 71199, 2391, 279, 8753, 22910, 323, 6244, 35414, 315, 9822, 304, 220, 5245, 19, 13, 1283, 374, 6646, 832, 315, 279, 12474, 6411, 20663, 304, 3925, 11, 3967, 369, 813, 39747, 35201, 323, 19092, 11692, 627), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738958657.986628, last_token_time=173895

In [ ]:
use_cached_outputs=False,

1 frames
/usr/local/lib/python3.11/dist-packages/vllm/entrypoints/llm.py in __init__(self, model, tokenizer, tokenizer_mode, skip_tokenizer_init, trust_remote_code, allowed_local_media_path, tensor_parallel_size, dtype, quantization, revision, tokenizer_revision, seed, gpu_memory_utilization, swap_space, cpu_offload_gb, enforce_eager, max_seq_len_to_capture, disable_custom_all_reduce, disable_async_output_proc, hf_overrides, mm_processor_kwargs, task, override_pooler_config, compilation_config, **kwargs)
    210             compilation_config_instance = None

ى max_num_seqs=16, وقول الارقام بن كم وكم

حسنًا، سأشرح لك معنى max_num_seqs=16 باللغة العربية:

معنى max_num_seqs=16:

هذه المعلمة تحدد الحد الأقصى لعدد المتواليات (sequences) التي يمكن للنموذج معالجتها في وقت واحد خلال الاستدلال (inference).

متوالية (sequence): في سياق معالجة اللغة الطبيعية، تشير المتوالية إلى سلسلة من الرموز (tokens) مثل الكلمات أو أجزاء الكلمات التي تُكوّن نصًا معينًا.
الاستدلال (inference): هي عملية استخدام النموذج المُدرّب لتوليد مخرجات أو تنبؤات بناءً على بيانات جديدة.
بمعنى آخر، يحدد max_num_seqs=16 أن النموذج يمكنه معالجة ست عشرة متوالية كحد أقصى في نفس الوقت. إذا كان لديك أكثر من ست عشرة متوالية، فسيتم تقسيمها إلى مجموعات أصغر ويتم معالجة كل مجموعة على حدة.

الهدف من تحديد max_num_seqs:

تحسين الأداء: معالجة عدة متواليات في وقت واحد يمكن أن يُسرّع من عملية الاستدلال بشكل ملحوظ، خاصةً عند استخدام أجهزة مثل وحدات معالجة الرسومات (GPUs).
التحكم في استخدام الذاكرة: معالجة عدد كبير جدًا من المتواليات في وقت واحد يمكن أن يؤدي إلى استهلاك كبير للذاكرة، مما قد يُسبب مشاكل في الأداء أو حتى تعطل البرنامج.
كتابة الأرقام:

ست عشرة تعادل ستة عشر.
مثال:

إذا كان لديك ثلاثون متوالية (ثلاثون) و max_num_seqs=16، فسيتم تقسيم المتواليات إلى مجموعتين:

المجموعة الأولى: ست عشرة متوالية (ستة عشر).
المجموعة الثانية: أربعة عشرة متوالية (أربعة عشر).
سيتم معالجة كل مجموعة على حدة، ثم يتم تجميع النتائج.

آمل أن يكون هذا الشرح واضحًا ومفيدًا لك!

### شغال

In [1]:
from awq import AutoAWQForCausalLM
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="float16",
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=10,
            swap_space=4)
sampling_params = SamplingParams(temperature=0.2, max_tokens=100)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 20:15:29 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 20:15:33 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 20:15:45 config.py:542] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward', 'score'}. Defaulting to 'generate'.
WARNING 02-07 20:15:45 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 20:15:45 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 20:15:45 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disa

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 20:16:15 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 20:16:17 worker.py:267] Memory profiling takes 1.27 seconds
INFO 02-07 20:16:17 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 20:16:17 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.14GiB; the rest of the memory reserved for KV Cache is 7.80GiB.
INFO 02-07 20:16:17 executor_base.py:110] # CUDA blocks: 4566, # CPU blocks: 2340
INFO 02-07 20:16:17 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 35.67x
INFO 02-07 20:16:22 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 7.13 seconds


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.28s/it, est. speed input: 1.87 toks/s, output: 23.35 toks/s]

[RequestOutput(request_id=0, prompt='Who is Napoleon Bonaparte?', prompt_token_ids=[128000, 15546, 374, 70527, 13789, 391, 20430, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution and became Emperor of France in 1804. He is considered one of the greatest military minds in history, known for his tactical genius and strategic leadership.\nNapoleon's early life was marked by his education at the École Militaire in Paris, where he graduated in 1785. He quickly rose through the ranks of the French army, becoming a general at the age of 24. During", token_ids=(70527, 13789, 391, 20430, 574, 264, 8753, 6411, 323, 5054, 7808, 889, 16392, 311, 71199, 2391, 279, 8753, 22910, 323, 6244, 35414, 315, 9822, 304, 220, 5245, 19, 13, 1283, 374, 6646, 832, 315, 279, 12474, 6411, 20663, 304, 3925, 11, 3967, 369, 813, 39747

In [1]:
from awq import AutoAWQForCausalLM
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

model = LLM(model="meta-llama/Llama-3.2-3B-Instruct",
            dtype="float16",
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=10,
            swap_space=4)
sampling_params = SamplingParams(temperature=0.2, max_tokens=100, do_sample=True)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 20:20:19 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 20:20:24 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 20:20:37 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'generate', 'classify', 'score'}. Defaulting to 'generate'.
WARNING 02-07 20:20:37 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 20:20:37 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 20:20:37 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disa

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 20:21:14 model_runner.py:1115] Loading model weights took 6.0160 GB
INFO 02-07 20:21:16 worker.py:267] Memory profiling takes 1.52 seconds
INFO 02-07 20:21:16 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
INFO 02-07 20:21:16 worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.14GiB; the rest of the memory reserved for KV Cache is 7.80GiB.
INFO 02-07 20:21:17 executor_base.py:110] # CUDA blocks: 4566, # CPU blocks: 2340
INFO 02-07 20:21:17 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 35.67x
INFO 02-07 20:21:23 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 8.61 seconds


TypeError: Unexpected keyword argument 'do_sample'

In [1]:
from awq import AutoAWQForCausalLM
import bitsandbytes as bnb
from vllm import LLM, SamplingParams # Import SamplingParams

model = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            dtype="float16",
            gpu_memory_utilization=0.95,
            max_model_len=2048,
            enforce_eager=True,
            max_num_seqs=10,
            swap_space=4)
sampling_params = SamplingParams(temperature=0.2, max_tokens=1)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 20:30:26 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 20:30:31 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 20:30:44 config.py:542] This model supports multiple tasks: {'classify', 'embed', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 02-07 20:30:44 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 20:30:44 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 20:30:44 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_paralle

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 20:31:50 model_runner.py:1115] Loading model weights took 14.2717 GB


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 369182 has 14.73 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 67.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
!rm -rf /root/.cache

In [2]:

from vllm import LLM, SamplingParams # Import SamplingParams
import os
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
model = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            dtype="float32",
            gpu_memory_utilization=1,
            cpu_offload_gb= 5,
            max_model_len=128,
            enforce_eager=True,
            max_num_seqs=30,
            swap_space=5)
sampling_params = SamplingParams(temperature=0.2, max_tokens=1)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 21:02:50 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

INFO 02-07 21:02:54 config.py:2378] Upcasting torch.bfloat16 to torch.float32.
INFO 02-07 21:03:07 config.py:542] This model supports multiple tasks: {'score', 'embed', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 02-07 21:03:07 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 21:03:08 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 21:03:08 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=128, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

INFO 02-07 21:03:11 cuda.py:179] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 02-07 21:03:11 cuda.py:227] Using XFormers backend.
INFO 02-07 21:03:12 model_runner.py:1110] Starting to load model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B...


OutOfMemoryError: CUDA out of memory. Tried to allocate 518.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 210.12 MiB is free. Process 22558 has 14.52 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 1.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
cpu_offload_gb
gpu_memory_utilization: float=0.9

In [1]:

from vllm import LLM, SamplingParams # Import SamplingParams
import os
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
model = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            dtype="float32",
            gpu_memory_utilization=0.8,
            cpu_offload_gb= 5,
            max_model_len=128,
            enforce_eager=True,
            max_num_seqs=20,
            swap_space=2)
sampling_params = SamplingParams(temperature=0.2, max_tokens=1)  # Temperature > 0 for sampling

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 21:06:08 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-07 21:06:10 config.py:2378] Upcasting torch.bfloat16 to torch.float32.
INFO 02-07 21:06:23 config.py:542] This model supports multiple tasks: {'embed', 'score', 'classify', 'reward', 'generate'}. Defaulting to 'generate'.
WARNING 02-07 21:06:23 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 21:06:23 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 21:06:23 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=128, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_

OutOfMemoryError: CUDA out of memory. Tried to allocate 518.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 210.12 MiB is free. Process 43333 has 14.52 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 1.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### aشغاااااااااااااال

In [1]:
from vllm import LLM, SamplingParams # Import SamplingParams
import os

# Try bfloat16 for lower precision
model = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            dtype="half",  # Changed to bfloat16
            gpu_memory_utilization=0.9, # Reduced
            cpu_offload_gb=3, # Reduced but unlikely to help
            max_model_len=128, # Reduced
            enforce_eager=True,
            max_num_seqs=10,  # Reduced
            swap_space=3) # Reduced
sampling_params = SamplingParams(temperature=0.2, max_tokens=1)

output = model.generate("Who is Napoleon Bonaparte?", sampling_params=sampling_params)

print(output)

INFO 02-07 21:17:02 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 21:17:05 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 21:17:20 config.py:542] This model supports multiple tasks: {'embed', 'score', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 02-07 21:17:20 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 21:17:20 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 21:17:20 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=128, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 21:18:44 model_runner.py:1115] Loading model weights took 11.2193 GB
INFO 02-07 21:18:47 worker.py:267] Memory profiling takes 2.38 seconds
INFO 02-07 21:18:47 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 21:18:47 worker.py:267] model weights take 11.22GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 0.72GiB; the rest of the memory reserved for KV Cache is 1.28GiB.
INFO 02-07 21:18:48 executor_base.py:110] # CUDA blocks: 1492, # CPU blocks: 3510
INFO 02-07 21:18:48 executor_base.py:115] Maximum concurrency for 128 tokens per request: 186.50x
INFO 02-07 21:18:51 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 6.54 seconds


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.46it/s, est. speed input: 12.26 toks/s, output: 1.53 toks/s]

[RequestOutput(request_id=0, prompt='Who is Napoleon Bonaparte?', prompt_token_ids=[151646, 15191, 374, 69427, 13481, 391, 19840, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' What', token_ids=(3555,), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738963131.6066134, last_token_time=1738963132.2790735, first_scheduled_time=1738963131.6480281, first_token_time=1738963132.2790735, time_in_queue=0.041414737701416016, finished_time=1738963132.299084, scheduler_time=0.00019846300006065576, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]


### شغال

In [1]:
from vllm import LLM, SamplingParams # Import SamplingParams
import os

# Try bfloat16 for lower precision
model = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            dtype="half",  # Changed to bfloat16
            gpu_memory_utilization=0.9, # Reduced
            cpu_offload_gb=3, # Reduced but unlikely to help
            max_model_len=128, # Reduced
            enforce_eager=True,
            max_num_seqs=10,  # Reduced
            swap_space=3) # Reduced
sampling_params = SamplingParams(temperature=0.2, max_tokens=50)

output = model.generate("Who is python?", sampling_params=sampling_params)

print(output)

INFO 02-07 21:20:52 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 02-07 21:20:55 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-07 21:21:08 config.py:542] This model supports multiple tasks: {'reward', 'generate', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
WARNING 02-07 21:21:08 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-07 21:21:08 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-07 21:21:08 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=128, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-07 21:22:38 model_runner.py:1115] Loading model weights took 11.2193 GB
INFO 02-07 21:22:41 worker.py:267] Memory profiling takes 2.54 seconds
INFO 02-07 21:22:41 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 02-07 21:22:41 worker.py:267] model weights take 11.22GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 0.72GiB; the rest of the memory reserved for KV Cache is 1.28GiB.
INFO 02-07 21:22:41 executor_base.py:110] # CUDA blocks: 1492, # CPU blocks: 3510
INFO 02-07 21:22:41 executor_base.py:115] Maximum concurrency for 128 tokens per request: 186.50x
INFO 02-07 21:22:44 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 6.32 seconds


Processed prompts: 100%|██████████| 1/1 [00:17<00:00, 17.33s/it, est. speed input: 0.29 toks/s, output: 2.89 toks/s]

[RequestOutput(request_id=0, prompt='Who is python?', prompt_token_ids=[151646, 15191, 374, 10135, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" I'm a bit confused.\n\nPython is a programming language. It's a high-level language, which means it's designed to be easy to read and write for humans. It's widely used for a variety of applications, including web development, data analysis", token_ids=(358, 2776, 264, 2699, 21815, 382, 30280, 374, 264, 15473, 4128, 13, 1084, 594, 264, 1550, 11591, 4128, 11, 892, 3363, 432, 594, 6188, 311, 387, 4135, 311, 1349, 323, 3270, 369, 12677, 13, 1084, 594, 13570, 1483, 369, 264, 8045, 315, 8357, 11, 2670, 3482, 4401, 11, 821, 6358), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1738963364.6417708, last_token_time=1738963382.0636744, first_scheduled_time=1738963364.7549582, first_token_time=17